In [162]:
import keras.callbacks
import tensorflow as tf
import numpy as np
from PIL import Image
import os
import random
import pandas as pd

## Todo:
1) use tf.data.Dataset to read from type dataset
2) use a dataset fn to apply a transformation such that it randomly selects an m no of images from n images of that pokemon 

In [141]:
class PokemonType(tf.data.Dataset):
    @staticmethod
    def _generator(dataset_dir: str):
        pokemon_types = os.listdir(dataset_dir)  # [type1, type2] -> class labels
        for pokemon_type in pokemon_types:
            for pokemon_name in os.listdir(os.path.join(dataset_dir, pokemon_type)):
                pokemon_paths = os.listdir(os.path.join(dataset_dir,pokemon_type,pokemon_name))
                for pokemon_path in pokemon_paths:
                    if not pokemon_path.endswith(b'.svg'):
                        pokemon_img_path=os.path.join(dataset_dir,pokemon_type,pokemon_name,pokemon_path)
                        pokemon_tensor=np.asarray(Image.open(pokemon_img_path).convert('RGB').resize((256,256)))
                        yield pokemon_tensor,pokemon_type

    def __new__(cls, dataset_dir: str):
        return tf.data.Dataset.from_generator(
            cls._generator,
            args=(dataset_dir,),
            output_signature=(
                tf.TensorSpec(shape=(256,256,3), dtype=tf.uint8),  # Assuming RGB images
                tf.TensorSpec(shape=(), dtype=tf.string)  # Class label dtype
            )
        )


In [262]:
class PokemonTypeDataset(tf.data.Dataset):
    @staticmethod
    def _generator(csv_dir):
        pokemon_df=pd.read_csv(csv_dir.decode())
        for index, pokemon in pokemon_df.iterrows():
            pokemon_path=pokemon["image_path"]
            if pokemon_path.endswith(".svg"):
                continue
            pokemon_tensor=np.asarray(Image.open(pokemon_path).convert('RGB').resize((256,256)))
            pokemon_tensor=pokemon_tensor/255.0
            pokemon_types=[]
            if not pd.isna(pokemon["type2"]):
                pokemon_types.append(pokemon["type2"])
            pokemon_types.append(pokemon["type1"])
            
            yield pokemon_tensor, np.asarray(pokemon_types)
    def __new__(cls, csv_dir:str):
        return tf.data.Dataset.from_generator(
            cls._generator,
            args=(csv_dir,),
            output_signature=(
                tf.TensorSpec(shape=(256,256,3), dtype=tf.float32),
                tf.TensorSpec(shape=(None,),dtype=tf.string)
            )
        ).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [258]:
dataset_dir="../new_dataset"
csv_dir="../csv/pokemon.csv"

In [259]:
dataset=PokemonType(dataset_dir)

In [260]:
dataset=PokemonTypeDataset(csv_dir).apply(tf.contrib.)

In [261]:
dataset_size=0
for element in dataset.as_numpy_iterator():
    dataset_size+=1

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot batch tensors with different shapes in component 1. First element had shape [1] and element 10 had shape [2]. [Op:IteratorGetNext] name: 

In [232]:
print(f"dataset size: {dataset_size}")

dataset size: 10667


In [233]:
'''
Test and training data
'''
shuffled_dataset=dataset.shuffle(buffer_size=dataset_size,seed=69) 

train_size=int(0.8*dataset_size)
test_size=dataset_size-train_size
train_ds=shuffled_dataset.take(train_size)
test_ds=shuffled_dataset.skip(train_size)

In [234]:
pokemon_types_len=len(os.listdir(dataset_dir))

In [235]:
pokemon_types_len

17

In [249]:
batch_size=32
steps_per_epoch=tf.math.ceil(train_size/batch_size)

In [238]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy')>=0.99:
            print('Test accuracy: %.2f%%' % (logs.get('accuracy')*100))
            self.model.stop_training = True

            

In [246]:
#MODEL BUILDING

def get_model():
    inputs=tf.keras.layers.Input(shape=(256,256,3))
    output=tf.keras.layers.Conv2D(32,(3,3),activation="relu")(inputs)
    output=tf.keras.layers.MaxPooling2D(pool_size=(2,2))(output)
    output=tf.keras.layers.Conv2D(64,(3,3),activation="relu")(output)
    output=tf.keras.layers.MaxPooling2D(pool_size=(2,2))(output)
    output=tf.keras.layers.Flatten()(output)
    output=tf.keras.layers.Dense(128,activation="relu")(output)
    output=tf.keras.layers.Dense(64,activation="relu")(output)
    output=tf.keras.layers.Dense(pokemon_types_len,activation="sigmoid")(output)
    model=tf.keras.models.Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
    return model

In [247]:
my_callback=MyCallback()
model=get_model()

In [252]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 246016)            0   

In [251]:
model.fit(x=train_ds,validation_data=test_ds,epochs=10,callbacks=[my_callback],batch_size=batch_size,use_multiprocessing=True,steps_per_epoch=steps_per_epoch)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\dhanu\anaconda3\envs\POKEDEX_CNN\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\dhanu\anaconda3\envs\POKEDEX_CNN\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\dhanu\anaconda3\envs\POKEDEX_CNN\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\dhanu\anaconda3\envs\POKEDEX_CNN\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\dhanu\anaconda3\envs\POKEDEX_CNN\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\dhanu\anaconda3\envs\POKEDEX_CNN\Lib\site-package

In [264]:
def gen():
    for i in range(5):
        yield [i]*i

for i,x in enumerate(gen()):
    print(i,x)

0 []
1 [1]
2 [2, 2]
3 [3, 3, 3]
4 [4, 4, 4, 4]


In [266]:
buff=[1]+[2]

In [268]:
arr=np.asarray([2,2])
arr.shape

(2,)

In [269]:
np.concatenate(arr)

ValueError: zero-dimensional arrays cannot be concatenated

In [270]:
buff+=[3,3,3]

In [271]:
buff

[1, 2, 3, 3, 3]